In [ ]:
!pip install langchain
!pip install -U langchain-community
!pip install openai
!pip install tiktoken
!pip install pypdf
!pip install pinecone-client
!pip install transformers torch sentence-transformers
!pip install datasets
!pip install faiss-cpu
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 

In [ ]:
import openai
import langchain
import pinecone
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
import PyPDF2
import os



In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

from sentence_transformers import SentenceTransformer
import torch

# Set up Hugging Face API token (if using Hugging Face Inference API)
HUGGING_FACE_API_KEY = "API_KEY"

# from dotenv import load_dotenv
# load_dotenv()

In [ ]:
# Load a pre-trained sentence transformer model for creating embeddings
sbert_model = SentenceTransformer('all-mpnet-base-v2')

# Example list of documents
documents = [
    "The capital of France is Paris.",
    "The Eiffel Tower is located in Paris.",
    "France is known for its cuisine and art.",
    "The capital of india is delhi.",
    "The NY is located in Paris.",
    "France is known cars."
]

# Create embeddings for the documents
document_embeddings = sbert_model.encode(documents, convert_to_tensor=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from torch.nn.functional import cosine_similarity

def retrieve_document(query):
    # Embed the user query
    query_embedding = sbert_model.encode(query, convert_to_tensor=True)

    # Compute cosine similarities
    similarities = cosine_similarity(query_embedding, document_embeddings)

    # Find the index of the most similar document
    best_match_idx = torch.argmax(similarities).item()
    return documents[best_match_idx]

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Choose a larger model like T5-3B or Flan-T5-XL
model_name = "google/flan-t5-xl"  # or "t5-3b" for T5-3B
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def load_context_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        context = ""
        for page in reader.pages:
            context += page.extract_text() + "\n"
    return context

def truncate_context(context, max_length=1024):
    # Tokenize the context
    tokens = tokenizer.tokenize(context)

    # Truncate to the maximum allowed length
    if len(tokens) > max_length:
        truncated_tokens = tokens[:max_length]
        truncated_context = tokenizer.convert_tokens_to_string(truncated_tokens)
    else:
        truncated_context = context

    return truncated_context

def generate_answer(question, context):
    # Truncate the context if it's too long
    truncated_context = truncate_context(context, max_length=1024)
    # Format the input correctly
    input_text = f"question: {question} context: {truncated_context}"
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=1024,
        truncation=True
    )

    # Generate the response with adjusted parameters for speed
    output = model.generate(
        **inputs,
        max_new_tokens=100,
        num_beams=1,  # Use 1 for faster inference
        early_stopping=True
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer

    # # Combine truncated context and question into a format that T5 understands
    # input_text = f"question: {question} context: {truncated_context}"

    # # Tokenize the input
    # inputs = tokenizer(input_text, return_tensors="pt", truncation=True)

    # # Generate the response
    # output = model.generate(**inputs, max_new_tokens=100, num_beams=5, early_stopping=True)
    # answer = tokenizer.decode(output[0], skip_special_tokens=True)
    # return answer


# -------------------------------------------------------------------------------------------------
# from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration

# # Load the tokenizer, retriever, and model
# tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-base")
# retriever = RagRetriever.from_pretrained("facebook/rag-token-base", index_name="exact")
# model = RagTokenForGeneration.from_pretrained("facebook/rag-token-base", retriever=retriever)

# def generate_answer(question):
#     # Tokenize the input question
#     inputs = tokenizer(question, return_tensors="pt")

#     # Generate the response using the RAG model
#     output = model.generate(**inputs)
#     answer = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
#     return answer
# -----------------------------------------------------------------------------------------------------
# # Load the RAG model and tokenizer
# from transformers import RagTokenForGeneration, RagTokenizer

# # Load the RAG model and tokenizer
# rag_model = RagTokenForGeneration.from_pretrained("facebook/rag-token-base")
# rag_tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-base")

# def generate_answer(question, context):
#     # Prepare the input with both question and context
#     inputs = rag_tokenizer(question, return_tensors="pt", context=context)

#     # Ensure the context is not None and properly formatted
#     if context is None or len(context.strip()) == 0:
#         raise ValueError("Context must not be empty or None.")

#     # Generate the response using the model
#     output = rag_model.generate(**inputs)
#     answer = rag_tokenizer.batch_decode(output, skip_special_tokens=True)[0]
#     return answer




tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:

def qa_chat(context):
    print("Google Flan: Ask me anything!")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            print("Google Flan Chatbot: Goodbye!")
            break

        # Generate an answer using the truncated document context
        response = generate_answer(user_input, context)
        print(f"Google Flan: {response}")

if __name__ == "__main__":
    # Load context from a file (change file path as needed)
    file_path = "test.pdf"  # Replace with the path to your text file
    context = load_context_from_pdf(file_path)

    # Start the chatbot with the loaded context
    qa_chat(context)


# def qa_chat():
#     print("RAG Chatbot: Ask me anything!")
#     while True:
#         user_input = input("You: ")
#         if user_input.lower() in ['exit', 'quit']:
#             print("RAG Chatbot: Goodbye!")
#             break

#         # Retrieve the most relevant document
#         context = retrieve_document(user_input)
#         print(f"[DEBUG] Context Retrieved: {context}")

#         # Generate an answer using the retrieved document
#         response = generate_answer(user_input,context)
#         print(f"RAG Chatbot: {response}")

# if __name__ == "__main__":
#     qa_chat()


Google Flan: Ask me anything!
Google Flan: The Theory of Evolution, proposed by Charles Darwin, posits that species evolve over time through natural selection. It explains how organisms adapt to their environments and how new species arise
